In [2]:
from __future__ import (unicode_literals, print_function, absolute_import,
                        division)


from ema_workbench import (Model, MultiprocessingEvaluator, Policy,
                           Scenario, IntegerParameter)
from ema_workbench.em_framework.evaluators import BaseEvaluator

from ema_workbench.em_framework.evaluators import perform_experiments
from ema_workbench.em_framework.optimization import (HyperVolume,
                                                     EpsilonProgress)
from ema_workbench.em_framework.samplers import sample_uncertainties
from ema_workbench.util import ema_logging
import time
from problem_formulation import get_model_for_problem_formulation
import pandas as pd
import seaborn as sns
from matplotlib import pyplot as plt


In [3]:
if __name__ == '__main__':
    ema_logging.log_to_stderr(ema_logging.INFO)

    model, steps = get_model_for_problem_formulation(3)

    reference_values = {'Bmax': 175, 'Brate': 1.5, 'pfail': 0.5,
                        'discount rate 0': 3.5, 'discount rate 1': 3.5,
                        'discount rate 2': 3.5,
                        'ID flood wave shape': 4}
    scen1 = {}

    for key in model.uncertainties:
        name_split = key.name.split('_')

        if len(name_split) == 1:
            scen1.update({key.name: reference_values[key.name]})

        else:
            scen1.update({key.name: reference_values[name_split[1]]})

    ref_scenario = Scenario('reference', **scen1)

In [4]:
#Run scenarios ansd policies random
with MultiprocessingEvaluator(dike_model) as evaluator:
        results = evaluator.perform_experiments(2, 2)

[MainProcess/INFO] pool started
[MainProcess/INFO] performing 2 scenarios * 2 policies * 1 model(s) = 4 experiments
[MainProcess/INFO] 1 cases completed
[MainProcess/INFO] 2 cases completed
[MainProcess/INFO] 3 cases completed
[MainProcess/INFO] 4 cases completed
[MainProcess/INFO] experiments finished
[MainProcess/INFO] terminating pool


In [ ]:
#Run optimization to find solutions in the pareto front
convergence_metrics = [EpsilonProgress()]

espilon = [1e3] * len(model.outcomes)
#espilon = [1000000 ,1, 1000000,1,  1000000,1,1000000,1,1000000,1, 1000000,1000000]   
 
nfe = 30000 # 200 proof of principle only, way to low for actual use

with MultiprocessingEvaluator(model) as evaluator:
        results, convergence = evaluator.optimize(nfe=nfe, searchover='levers',
                                                  epsilons=espilon,
                                                  convergence=convergence_metrics,
                                                  reference=ref_scenario)

[MainProcess/INFO] pool started
[MainProcess/INFO] generation 0: 0/30000 nfe
[MainProcess/INFO] generation 5: 499/30000 nfe
[MainProcess/INFO] generation 10: 996/30000 nfe
[MainProcess/INFO] generation 15: 1494/30000 nfe
[MainProcess/INFO] generation 20: 1992/30000 nfe
[MainProcess/INFO] generation 25: 2489/30000 nfe
[MainProcess/INFO] generation 30: 2989/30000 nfe
[MainProcess/INFO] generation 35: 3489/30000 nfe
[MainProcess/INFO] generation 40: 3988/30000 nfe
[MainProcess/INFO] generation 45: 4486/30000 nfe
[MainProcess/INFO] generation 50: 4985/30000 nfe
[MainProcess/INFO] generation 55: 5482/30000 nfe
[MainProcess/INFO] generation 60: 5978/30000 nfe
[MainProcess/INFO] generation 65: 6476/30000 nfe
[MainProcess/INFO] generation 70: 6973/30000 nfe
[MainProcess/INFO] generation 75: 7470/30000 nfe
[MainProcess/INFO] generation 80: 7968/30000 nfe
[MainProcess/INFO] generation 85: 8466/30000 nfe
[MainProcess/INFO] generation 90: 8962/30000 nfe
[MainProcess/INFO] generation 95: 9461/30000

In [1]:
fig, (ax1, ax2) = plt.subplots(ncols=2, sharex=True, figsize=(8,4))
ax1.plot(convergence.nfe, convergence.epsilon_progress)
ax1.set_ylabel('$\epsilon$-progress')
ax1.set_xlabel('number of function evaluations')
plt.show()

NameError: name 'plt' is not defined

In [7]:
model.outcomes.keys()

odict_keys(['A.1 Total Costs', 'A.1_Expected Number of Deaths', 'A.2 Total Costs', 'A.2_Expected Number of Deaths', 'A.3 Total Costs', 'A.3_Expected Number of Deaths', 'A.4 Total Costs', 'A.4_Expected Number of Deaths', 'A.5 Total Costs', 'A.5_Expected Number of Deaths', 'RfR Total Costs', 'Expected Evacuation Costs'])

In [21]:
#Sum the different RfR values to identify the one with 0 for the our municiaplites. identify intresting solutions
results ["sum rfr"] = (results ['0_RfR 0'] + 
                       results ['0_RfR 1'] +
                       results ['0_RfR 2'] +
                       results ['1_RfR 0'] + 
                       results ['1_RfR 1'] +
                       results ['1_RfR 2'] +
                       results ['2_RfR 0'] + 
                       results ['2_RfR 1'] +
                       results ['2_RfR 2'])




results ["sum deaths"] = (results ['A.1_Expected Number of Deaths'] + 
                       results ['A.2_Expected Number of Deaths'] +
                       results ['A.3_Expected Number of Deaths'] +
                       results ['A.4_Expected Number of Deaths'] + 
                       results ['A.5_Expected Number of Deaths'])
                     


int_sol = results[results["sum rfr"] <= 0 ] #no RFR costs for dikes
int_solu = int_sol[int_sol['A.1_Expected Number of Deaths'] <= 0 ] #no deaths in dike 1
int_solut = int_solu[int_solu['A.2_Expected Number of Deaths'] <= 0.00148 ] #0.00148 for dike 2
int_solutions = int_solut[int_solut['A.3_Expected Number of Deaths'] <= 0.001 ] #0.001 for dike 3
# int_solutions

int_solutions


,0_RfR 0,0_RfR 1,0_RfR 2,1_RfR 0,1_RfR 1,1_RfR 2,2_RfR 0,2_RfR 1,2_RfR 2,3_RfR 0,...,A.3 Total Costs,A.3_Expected Number of Deaths,A.4 Total Costs,A.4_Expected Number of Deaths,A.5 Total Costs,A.5_Expected Number of Deaths,RfR Total Costs,Expected Evacuation Costs,sum rfr,sum deaths
118,0,0,0,0,0,0,0,0,0,0,...,3.422156e+07,0.000000,1.317228e+07,0.003736,1.335788e+08,0.000000,0.0,0.00000,0,0.003736
522,0,0,0,0,0,0,0,0,0,0,...,4.986145e+07,0.000925,1.811829e+07,0.002690,1.242471e+08,0.000000,0.0,511.92063,0,0.003616
707,0,0,0,0,0,0,0,0,0,0,...,7.200683e+07,0.000000,2.157461e+07,0.003399,5.407116e+07,0.032159,121200000.0,0.00000,0,0.035558


In [23]:
#Save results
from ema_workbench import save_results

save_results(int_solutions, 'results/ identified solutions MORDM.tar.gz')

ValueError: too many values to unpack (expected 2)

In [ ]:
#Load results

from ema_workbench import load_results

int_solutions = load_results('results/{} identified solutions MORDM.tar.gz')

In [26]:
#identified the lever values that provided those solutions
policies = int_solutions
policies = policies.drop([o.name for o in model.outcomes], axis=1)
policies = policies.drop([ "sum rfr", "sum deaths"], axis=1)
policies

,0_RfR 0,0_RfR 1,0_RfR 2,1_RfR 0,1_RfR 1,1_RfR 2,2_RfR 0,2_RfR 1,2_RfR 2,3_RfR 0,...,A.2_DikeIncrease 2,A.3_DikeIncrease 0,A.3_DikeIncrease 1,A.3_DikeIncrease 2,A.4_DikeIncrease 0,A.4_DikeIncrease 1,A.4_DikeIncrease 2,A.5_DikeIncrease 0,A.5_DikeIncrease 1,A.5_DikeIncrease 2
118,0,0,0,0,0,0,0,0,0,0,...,1,7,0,0,0,2,0,7,2,6
522,0,0,0,0,0,0,0,0,0,0,...,4,3,2,0,0,0,1,7,2,4
707,0,0,0,0,0,0,0,0,0,0,...,0,3,1,2,0,0,8,0,0,1


In [27]:
#From the identified levers create a Policy dictionary to be fed to the Multiprocess evaluator
policies_to_evaluate = []

for i, policy in policies.iterrows():
    policies_to_evaluate.append(Policy(str(i), **policy.to_dict()))

# policies_to_evaluate

In [56]:
#Run scenarios ansd policies random
with MultiprocessingEvaluator(model) as evaluator:
        results = evaluator.perform_experiments(5, policies_to_evaluate)

[MainProcess/INFO] pool started
[MainProcess/INFO] performing 5 scenarios * 32 policies * 1 model(s) = 160 experiments
[MainProcess/INFO] 16 cases completed
[MainProcess/INFO] 32 cases completed
[MainProcess/INFO] 48 cases completed
[MainProcess/INFO] 64 cases completed
[MainProcess/INFO] 80 cases completed
[MainProcess/INFO] 96 cases completed
[MainProcess/INFO] 112 cases completed
[MainProcess/INFO] 128 cases completed
[MainProcess/INFO] 144 cases completed
[MainProcess/INFO] 160 cases completed
[MainProcess/INFO] experiments finished
[MainProcess/INFO] terminating pool


In [ ]:
#Save results
from ema_workbench import save_results

save_results(int_solutions, 'results/ identified solutions MORDM 1000scenarios.tar.gz')